In [ ]:
#https://python.langchain.com/v0.2/docs/how_to/MultiQueryRetriever/

In [ ]:
#invoke tracing via phoenix
from phoenix.trace.langchain import LangChainInstrumentor

LangChainInstrumentor().instrument()

In [ ]:
import weaviate

client = weaviate.Client("http://localhost:8081")
weaviate_client = weaviate.connect_to_local("localhost","8081")#v4
index_name="Phoenix_test"

In [ ]:
weaviate_client.collections.delete_all()
weaviate_client.collections.list_all()

In [ ]:
from weaviate.classes.config import Configure
from weaviate.classes.config import Property, DataType

weaviate_client.collections.create(
    index_name,
    #see notes above re: the docker modules that need to be enabled for text2vec* to work correctly -e ENABLE_MODULES=text2vec-ollama
    vectorizer_config=Configure.Vectorizer.text2vec_ollama( 
        model="nomic-embed-text",    
        api_endpoint="http://host.docker.internal:11434",
    ),
    # generative_config=Configure.Generative.ollama(
    #     api_endpoint = "http://host.docker.internal:11434",
    #     model="jonphi"
    # ),

    # properties=[
    #     Property(name="page_content", data_type=DataType.TEXT),
    #     Property(name="source", data_type=DataType.INT),
    # ]

)

In [19]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
import pprint


#todo - look into keeping bullets together in available loader libraries
loader = TextLoader("/home/jonot480/Documents/paulgraham.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=200)
docs = text_splitter.split_documents(documents)
#print(*docs, sep='\n')

In [20]:
from langchain_community.vectorstores.weaviate import Weaviate
from langchain_community.embeddings.ollama import OllamaEmbeddings

vectordb = Weaviate.from_documents(
    docs, 
    embedding=OllamaEmbeddings(model='nomic-embed-text'),
    client=client,    
    index_name=index_name,
    #prefer_grpc=True, 
) 


In [ ]:
collection = weaviate_client.collections.get(index_name)
response = collection.query.fetch_objects()

for o in response.objects:
    print(o.properties)  # Inspect returned objects
    #print(o.properties["text"])


In [22]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.chat_models import ChatOllama

question = "What is the effective date?"
llm = ChatOllama(model="phi3",temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [23]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)



In [ ]:
unique_docs = retriever_from_llm.invoke(question)
len(unique_docs)

In [ ]:
from typing import List
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


# Output parser will split the LLM result into a list of queries
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return lines


output_parser = LineListOutputParser()


question = "What is the effective date?"


QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

#langchain's code doesn't work (see section below, so i used this to get around it)
#https://github.com/langchain-ai/langchain/issues/17352
retriever_from_llm = MultiQueryRetriever.from_llm( retriever=vectordb.as_retriever(), llm=llm, prompt=QUERY_PROMPT )

unique_docs = retriever_from_llm.invoke("What is the effective date?")
len(unique_docs)

In [ ]:
for i in unique_docs:
    print(i)
#print(unique_docs)

In [ ]:
#this is langchain's code but it doesn't work 
#https://python.langchain.com/v0.2/docs/how_to/MultiQueryRetriever/
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(), llm_chain=llm_chain
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.invoke("What does the course say about regression?")
len(unique_docs)